<!-- Python notebook on basic linear control: negative feedback loops, feedforward terms, PID control, LQR and basic Lyapunov analysis. -->

# Robot control systems

Robots are equipped with various actuators that receive inputs and sensors that provide outputs. A robot's brain (its computer) faces the task of deciding what signals to send to the actuators based on the measured sensor readings such that a robot executes desired behavior. This task of controlling the robot must address various challenges such as measurement noise, inaccurate robot models, non-ideal actuators and  external disturbances. Control theory offers a systematic approach to address this challenge by designing control systems to manage robot behavior. 


<img src="./media/system.png" alt="A system" width="500"/>



A robot can be modelled as a _system_ taking inputs $u(t)$ and producing outputs $y(t)$, such that $y(t) = h(u(t))$. 

Let us start with a simple, yet widely-applicable class of systmes, namely a discrete-time linear time invariant (LTI) system. It can be expressed the the so-called state-space form as follows

$x_{k+1} = Ax_k + Bu_k$,

and the output is also given by a linear equation

$y_k = Cx_k + Du_k$.

The matrix $D = 0$ most often in practice. The system is called linear because the input-output relationship is affine, and time-invariant because the dynamics parameters $A, B, C$, and $D$ are assumed to not depend on time.

#### Exercise 3.1: 

Consider a point-mass object of mass $m$ is free to move along x-axis. The only force acting on it is the external force $f$ in the x-direction. 

$m\ddot{x} = f$

Consider the position and velocity of the object as the states and the $f$ as the control input, x-position as the output. Use forward Euler integration scheme with timestep $T_s$ for discretizing the system. Provide the $A,B,C,$ and $D$ matrices associated with this system.

Answer: $A = \begin{bmatrix} 1 & T_s \\ 0 & 1 \end{bmatrix} $, $B = \begin{bmatrix} 0 \\ \frac{T_s}{m} \end{bmatrix}$, $C = \begin{bmatrix} 1 & 0 \end{bmatrix}$ and $D = \begin{bmatrix} 0 \end{bmatrix}$.

## Trajectory tracking

Consider the trajectory-tracking task, where the system should controlled such that its output follows a desired trajectory $[y_{\mathrm{des},0}, y_{\mathrm{des},1}, y_{\mathrm{des},2}, y_{\mathrm{des},3}, ... ]$. One of the simplest controllers that can try to execute this task is an `open-loop' feedforward controller shown below.

<img src="./media/FF.png" alt="Feedforward control system" width="500"/>

Here, the feedforward controller computes the appropriate control input to apply depending on the desired system output. It uses knowledge of the starting state of the system and inverts the system dynamics to compute the trajectory of control inputs to be applied. For example, for a system at state $x_k$ the linear equation below can be solved for $u_k$ to compute the feedforward control action for trajectory following. 

$y_{\mathrm{des},k+1} = C(Ax_k + Bu_k) + Du_k$.

Pros of pure feedforward control: 
- FF control systems are simple.
- No sensing and estimation are required.
- Can follow complex desired trajectories.

Cons of pure feedforward control:
- Rarely works in practice.
- Sensitive to initial state estimation error and model errors.
- Inverting system dynamics may not always be possible.

### Exercise 3.2:
For the point mass system derived in exercise 3.1 and the desired x-positions given below, compute the feedforward control actions. (Hint: Dynamics not directly invertible for tracking position, try tracking the velocity.)

In [ ]:
import pinocchio as pin
import time
import numpy as np
from numpy.linalg import inv,norm,pinv,svd,eig
from scipy.optimize import fmin_bfgs,fmin_slsqp
from scipy.linalg import solve_discrete_are
import matplotlib.pylab as plt

In [ ]:
np.random.seed(0)
# track a sine function
frequency = 2
Ts = 0.01
m = 1

y_des = np.array([np.sin(2*np.pi*frequency*i*Ts) for i in range(200)])

# define an LTI system class
class LTI_system:
    def __init__(self,A,B,C=None,D=None):
        self.A = A
        self.B = B
        if C is None:
            self.C = np.eye(A.shape)
        else:
            self.C = C
        if D is None:
            self.D = np.zeros((C.shape[0],B.shape[1]))
        else:
            self.D = D
        
    def step(self,x,u):
        return self.A @ x + self.B @ u
    
    def output(self,x,u):
        return self.C @ x + self.D @ u
    
    def simulate(self,x0,u_traj):
        x_step = self.step(x0,u_traj)
        y_step = self.output(x0,u_traj)
        return x_step, y_step

u_traj_ff = []
y_traj = []
x_traj = []
x0 = np.zeros((2,))

x_traj.append(x0)



## Your code goes below
## Use the LTI class above to initialize point-mass system from Ex 3.1, compute the feedforward actions and simulate the system. 

# Compute desired velocities using FD
v_des = np.array([(y_des[i+1] - y_des[i])/Ts for i in range(199)])
v_des = np.append(v_des, v_des[-1]) # Use backward FD for last value

# Compute feedforward control actions from desired velocities using FD
a_des = np.array([(v_des[i+1] - v_des[i])/Ts for i in range(199)])
a_des = np.append(a_des, a_des[-1]) # Use backward FD for last value
u_traj_ff = m*a_des

# You can also compute feedforward actions using analytical derivatives instead of finite differences. This is also accepted.

A = np.array([[1 , Ts], [0, 1]])
B = np.array([[0], [Ts/m]])
C = np.array([[1, 0]])
D = np.array([[0]])

sys = LTI_system(A,B,C,D)

# Part below is optional, not required by the assignment

match_initial_conditions = False # Set this to True to also see how feedforward is sensitive to initial conditions. 

if match_initial_conditions:
    x0[1] = (y_des[1] - y_des[0])/Ts # Note: initial condition for velocity should have been set to match the sine function. But the released assignment missed this line. So to get accurate tracking, one option could have been to set solve for the first two control inputs to get correct states after two time-steps. But this is not strictly required for the assignment.
else:
    # One systematic approach to compute the feedforward is to consider the `controllability matrix' of the system to find actions that will drive the system to the desired trajectory.
    #x[2] = A*A*x[0] + A*B*u[0] + B*u[1] 
    #[u[0], u[1]] = inv(np.hcat[A*B, B]) * (x[2] - A*A*x[0]) 
    # x0 = [0, 0] as stated in the code. Setting x[2] to desired trajectory gives x[2] = [y_des[2], v_des[2]] 

    controllability_matrix = np.hstack((sys.A @ sys.B, B))
    u_12 = np.linalg.inv(controllability_matrix) @ (np.array([y_des[2], v_des[2]]) -A @ A @ x0)

    # You could have considered the initial conditions some other way to compute the feedforward. If you have not, you will find that simulating with the feedforward will cause the trajectory to drift. This is okay. It would also illustrate the sensitivity of pure feedforward control to initial conditions.
    u_traj_ff[0] = u_12[0]
    u_traj_ff[1] = u_12[1]

# You might have used A,B,C,D matrices to compute the feedforward signals. This is also accepted.

print(u_12) # You will find the computed feedforward actions here to result in large control inputs, that will likely exceed the actuator limits. Requiring the state to exactly match the desired trajectory directly after two time steps is often too stringent a requirement.



In [ ]:
# plot control actions
plt.figure()
plt.plot(u_traj_ff)

In [ ]:
# a = C@x_traj[-1] + D@u_ff
# x_traj

In [ ]:
for i in range(len(y_des)):
    x_step, y_step = sys.simulate(x_traj[-1], np.array([u_traj_ff[i]]))
    x_traj.append(x_step)
    y_traj.append(y_step)    

In [ ]:
# plot the simulated trajectory and the desired trajectory

plt.figure()
plt.plot(y_des)
plt.plot(y_traj)
plt.legend(['desired','simulated'])
plt.xlabel('time (s)')
plt.ylabel('position (m)') 

plt.show()

### Exercise 3.3: 
Suppose that the mass of the system is not exactly known and that the true mass is 0.9 times the previous mass. Create a new system with this true mass and compute y_traj_real.

In [ ]:
#Your code here
y_traj_real = []

m_new = 0.9
B_new = np.array([[0], [Ts/m_new]])
sys_new = LTI_system(A,B_new,C,D)

x_traj_real = []
x_traj_real.append(x0)
y_traj_real = []    

for i in range(len(y_des)):
    x_step, y_step = sys_new.simulate(x_traj_real[-1], np.array([u_traj_ff[i]]))
    
    x_traj_real.append(x_step)
    y_traj_real.append(y_step)

plt.figure
plt.plot(y_des)
plt.plot(y_traj_real)
plt.plot(y_traj)
plt.legend(['desired','real','feedforward'])
plt.xlabel('time (s)')
plt.ylabel('y (m)')
plt.show()

## State Feedback control

It is clear from the previous plot that the feedforward control is not enough to track the desired trajectory in the presence of noise, model errors and disturbances. We need a negative feedback loop that measures the error between the desired trajectory and the real trajectory and uses it to reduce tracking errors. Negative feedback loops, being widely and frequently used, are nearly synonymous with the field of control.

<img src="./media/FB.png" alt="Feedback control system" width="500"/>

For LTI systems, a common form of negative feedback is the linear state feedback law 

$u = K (x_\mathrm{des} - x)$.

If the control task involves taking the system to the origin ($x_\mathrm{des} = 0$), the closed-loop system is then described by the following discrete-time LTI system:

$x_{k+1} = (A - BK)x_k$

$y_k = Cx_k$

Typically, state estimators (e.g., [Kalman Filter](https://en.wikipedia.org/wiki/Kalman_filter)) are used to estimate the value of $x_k$ from past sensor readings $y_0, \dots, y_{k}$, to be able to apply state-feedback. However, for the rest of this assignment, let us assume that the state of our given system is directly measured, meaning that $C = I_{2\times 2}$.



### Exercise 3.4
For the point-mass system, design a state feedback controller for tracking the desired trajectory. Use zero for the desired velocity. (Hint: The design involves guessing the matrix K)

In [ ]:
y_traj_fb = []
x_traj_fb = []
x_traj_fb.append(x0)

y_des.shape
x_des = np.vstack([y_des,y_des*0]) # Note that desiring zero velocity for the point-mass is inconsistent with it executing a sinusoidal motion. The ensuing controller with this design choice would be a 'regulator' and not a tracking controller. Typical examples of regulators in real life are cruise control in cars, or thermo-regulation in a room. Attempting to track a sinusoidal trajectory with a regulator would result in a non-zero steady state error and is an educational exercise in tuning and understanding the limitations of regulators.
#Setting x_des as below would have been a better choice for tracking a sinusoidal trajectory. You will be able to use smaller gains that are less likely to saturate actuators. Try it out to see how much better the controller behaves. In your future robotics projects, if you have a PD controller do not forget to send reference velocities to the controller. Tracking higher derivatives of the desired position reference is key for high performance tracking.
# x_des = np.vstack([y_des,v_des]) # with this choice, the controller gains can be lower.

K = np.array([[5000.0, 100.0]]) # But these gains are quite high

u_traj_fb = []

for i in range(200):

    # pass
    x_err = x_des[:,i] - x_traj_fb[-1]
    u = K @ x_err
    u_traj_fb.append(u)
    x_step, y_step = sys.simulate(x_traj_fb[-1], u)
    x_traj_fb.append(x_step)
    y_traj_fb.append(y_step[0])


In [ ]:
# Plot state tracking
plt.figure()
plt.plot(y_des)
plt.plot(y_traj_fb)
plt.legend(['desired','simulated'])
plt.xlabel('time (s)')
plt.ylabel('position (m)')
plt.show()

In [ ]:
# plot control actions
plt.figure()
plt.plot(u_traj_fb)
plt.plot(u_traj_ff)
plt.legend(['feedback','feedforward'])

## Exercise 3.5
Suppose that the system also has noisy dynamics. Add Gaussian noise to the x_step output of the LTI system. Simulate the system you designed in Ex 3.4. What can you say about its behavior and how the magnitude of $K$ affects tracking in noisy systems.

In [ ]:
y_traj_fb = []
x_traj_fb = []
x_traj_fb.append(x0)

# K = np.array([[5000.0, 100]]) # But these gains are quite high

u_traj_fb = []

for i in range(200):

    # pass
    x_err = x_des[:,i] - x_traj_fb[-1]
    u = K @ x_err
    u_traj_fb.append(u)
    x_step, y_step = sys.simulate(x_traj_fb[-1], u)
    x_step += np.random.normal(0, 0.02, x_step.shape)
    x_traj_fb.append(x_step)
    y_traj_fb.append(y_step[0])

In [ ]:
# Plot state tracking
plt.figure()
plt.plot(y_des)
plt.plot(y_traj_fb)
plt.legend(['desired','simulated'])
plt.xlabel('time (s)')
plt.ylabel('position (m)') # Disturbance rejection is quite good since high gains are used.
plt.show()

In [ ]:
# plot control actions
plt.figure()
plt.plot(u_traj_fb)
plt.legend(['feedback']) # Because high control gains were chosen to reduce tracking error, the influence of the measurement noise on control actions is quite significant. Typically actuators cannot provide such noisy control actions and it might even be unsafe to send this control signal. It is a common trade-off in control design between tracking performance and robustness to noise.

Superior tracking often requires both feedforward and feedback control as shown in 

<img src="./media/FF_FB.png" alt="Feedback control system" width="700"/>

Feedforward terms permit tracking complex trajectories using the knowledge of system dynamics while the feedback terms stabilize the system around the desired setpoint.

### Exercise 3.6
Combine the feedback and feedforward actions to track the desired trajectory for the point mass system from Exercise 3.3.

In [ ]:
y_traj_fbff = []
x_traj_fbff = []
u_traj_fb = []
u_traj = []
x_traj_fbff.append(x0)

for i in range(200):
    x_err = x_des[:,i] - x_traj_fbff[i]  
    u = K @ x_err + u_traj_ff[i]
    u_traj_fb.append(K @ x_err)
    u_traj.append(u)
    x_step, _ = sys_new.simulate(x_traj_fbff[-1], u_traj[-1])
    x_traj_fbff.append(x_step)
    y_traj_fbff.append(sys_new.output(x_step, u_traj[-1]))


In [ ]:
# plot the simulated trajectory and the desired trajectory

plt.figure()
plt.plot(y_des)
plt.plot(y_traj_real)
plt.plot(y_traj_fbff)
plt.legend(['desired','feedforward','feedback+feedforward'])
plt.xlabel('time (s)')
plt.ylabel('position (m)')
plt.show()

In [ ]:
# plot control actions
plt.figure()
plt.plot(u_traj_fb)
plt.plot(u_traj_ff)
plt.legend(['feedback','feedforward']) # Feedforward is doing most of the work, feedback is just correcting for the tracking error due to the model mismatch. 

## System stability

You perhaps encountered system instability (where the trajectories keep growing in magnitude) and are curious if there is a systematic approach to define and check for this phenomenon.

Let us consider regulation problem, where the desired trajectory being tracked is a fixed set-point. Without loss of generality (why?) for regulation problems, we can assume that we want the system to reach the origin. A system 

$x_{k+1} = A x_k$,

is said to be asymptotically stable if $\lim_{k \to \infty} x_k = 0$. For an LTI system, checking for stability amounts to checking if the magnitude of all the eigenvalues of the matrix $A$ is strictly less than 1.0. Then the system is said to be strictly stable. If one eigenvalue has a magnitude equal to 1, the system is said to be marginally stable. If more than one eigenvalue has a magnitude equal to 1, the system may be unstable. If even one eigenvalue magnitude is strictly greater than one, the system is unstable.

Note that a closed loop LTI system is given by

$x_{k+1} = A_\mathrm{cl} x_k$,

where $A_\mathrm{cl} = A - B K$. Therefore, stability of the closed-loop LTI systems is characterized using the eigenvalues of the matrix $A_\mathrm{cl}$. 

It is important to characterize the stability of the feedback system, as it is a property that implies 'good behavior' of the closed-loop system, i.e. that it will not diverge or oscillate from the desired set-point.  Unstable systems can be dangerous in practice. 


### Exercise 3.7 

Compute the eigenvalues of the point-mass system and of the closed-loop system with the linear feedback gains chosen in Exercise 3.4. Characterize the stability property of these two systems.

Answer: Enter your answer here.

In [ ]:
A_cl = A - B @ K
eigvals, eigvecs = eig(A_cl)
print(eigvals)

# compute the magnitude of the eigenvalues
eigvals_mag = np.abs(eigvals)
print(eigvals_mag)


Since the magnitudes of both the eigenvalues are strictly below one, the closed-loop system for the hand-tuned control gains is stable.

While it was fairly straightforward to guess stable feedback gains for the point mass system, this is not so easy for more complex systems with higher dimensions. Fortunately, there exist several methods to compute these gains. One of the most popular methods is Linear Quadratic Regulator (LQR). LQR is a method that minimizes the cost function, (assuming that the system is being stabilized around the origin):

$J = \sum_{t=0}^{T} (x_t^T Q x_t + u_t^T R u_t)$,
where $Q$ and $R$ are positive definite matrices. The LQR controller is the optimal feedback gain that minimizes this cost function. The LQR controller can be computed using the following steps:

 1. Solve the discrete-time Riccati equation for the system:
 
    $ P = A^T P A - A^T P B (B^T P B + R)^{-1} B^T P A + Q $ 

2. Compute feedback gain

    $K = (R + B^T P B)^{-1} B^T P A$

3. The control law is then:
    
    $ u = -K x $

LQR (and linear control in general) is a useful tool even for controlling nonlinear systems. Nonlinear systems are often locally linearizable, using their first derivatives. The controller design is then performed for this linearized system and can be effective as long as the nonlinear system remains close to the linearization point.

In this exercise, we will consider the nonlinear system cart-pole, linearized around the upright position. The state of the system is 

$x = \begin{bmatrix} x \\ \theta \\ \dot{x} \\ \dot{\theta} \end{bmatrix}$.

The system dynamics and the linearization around the upright position can be found in 
https://courses.ece.ucsb.edu/ECE594/594D_W10Byl/hw/cartpole_eom.pdf

Assume that $M = 1$, $m_p = 1$ and $L = 1$.

### Exercise 3.8 

Compute the LQR feedback gains for the cart-pole system linearized at the upright position. Stabilize the system at the upright position the LQR controller, starting from a state close to the upright position. Simulate the nonlinear system for 10 seconds and plot the state trajectories.

(Hint: use scipy.linalg.solve_discrete_are to solve the Riccati equation)

In [ ]:
g = 9.81
M = 1
m = 1
l = 1

# Define the system matrices
A = np.array([[1, Ts, 0, 0], [0, 1, Ts*g, 0], [0, 0, 1, Ts], [0, 0, 2*g*Ts, 1]])
B = np.array([[0], [Ts], [0], [Ts]])

C = np.array([[0, 0, 1, 0], [0, 0, 0, 1]])
D = np.zeros((2,1))

# Define the system
sys = LTI_system(A,B,C,D)

# compute LQR feedback gain
Q = np.diag([0.1,0.1,1,1])
R = np.diag([1])
P = solve_discrete_are(A,B,Q,R)
K = inv(R + B.T @ P @ B) @ B.T @ P @ A

In [ ]:
# nonlinear system, Forward Euler
def nls_cartpole(x,u, Ts):
    M = 1
    m = 1
    l = 1
    xcdot = x[1]
    thetadot = x[3]
    xc = x[0]
    theta = x[2]
    s = np.sin(theta)
    c = np.cos(theta)
    u = np.clip(u, -5, 5)
    xcddot = (-s*(m*l)*c*thetadot**2 + s*m*c*g + u)/(-m*c**2 + M + m)
    thetaddot = (-s*(m*l)*c*thetadot**2 + s*g*(M+m) + u*c)/(l*(-m*c**2 + M + m))
    xc = xc + Ts*xcdot
    xcdot = xcdot + Ts*xcddot
    theta = theta + Ts*thetadot
    thetadot = thetadot + Ts*thetaddot
    return np.hstack([xc,xcdot,theta,thetadot])

In [ ]:
Nsteps = 1000
# simulate the system
x0 = np.array([0,0,0.1,0])
x_traj = []
x_traj.append(x0)
u_traj = []
y_traj = []
y_traj.append(C @ x0)
for i in range(Nsteps):
    u = -K @ x_traj[-1]
    x_step = nls_cartpole(x_traj[-1],u,Ts)
    x_traj.append(x_step)
    y_traj.append(C @ x_step)
    u_traj.append(u)


In [ ]:

# plot the simulated trajectory and the desired trajectory
plt.figure()
plt.plot(np.linspace(0,Nsteps*Ts,Nsteps+1),[y[0] for y in y_traj])
plt.plot(np.linspace(0,Nsteps*Ts,Nsteps+1),[y[1] for y in y_traj])
plt.legend(['$\\theta$','$\\dot{\\theta}$'])
plt.xlabel('time (s)')
plt.ylabel('position (m)')
plt.show()


In [ ]:

# plot the control input
plt.figure()
plt.plot(np.linspace(0,Nsteps*Ts,Nsteps),u_traj)
plt.xlabel('time (s)')
plt.ylabel('control input')
plt.show()